<a href="https://colab.research.google.com/github/Rylan-Lewis/Machine-Learning/blob/main/ResNet-50.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
from torch.utils.data import DataLoader
import torch.optim as optim
from torch.optim import lr_scheduler
from torchvision.datasets import ImageFolder
from torch.utils.data import random_split, DataLoader
from PIL import Image
import os
import torch.nn.functional as F

In [ ]:
#We are now making the blocks(set of 3 layers) which will be used throughout the model
class block(nn.Module):
    def __init__(self,in_channels,out_channels,identity_downsample=None , stride=1):
        super(block,self).__init__()
        self.expansion = 4
        self.conv1 = nn.Conv2d(in_channels,out_channels,kernel_size=1, stride=1,padding=0)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels,out_channels,kernel_size=3,stride=stride,padding=1)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.conv3=nn.Conv2d(out_channels,out_channels*self.expansion,kernel_size=1,stride=1,padding=0)
        self.bn3 = nn.BatchNorm2d(out_channels*self.expansion)
        self.relu = nn.ReLU()
        self.identity_downsample = identity_downsample


    def forward(self,x):
        identity=x

        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu(x)
        x = self.conv3(x)
        x = self.bn3(x)

        if self.identity_downsample is not None:
            identity=self.identity_downsample(identity)

        x += identity
        x = self.relu(x)
        return x

In [ ]:
class ResNet(nn.Module):
    def __init__(self,block,layers,image_channels,num_classes):
        super(ResNet,self).__init__()
        self.in_channels = 64
        self.conv1 = nn.Conv2d(image_channels,64,kernel_size=7,stride=2,padding=3)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2,padding=1)

        #resNet layers
        self.layer1 = self.make_layer(block,layers[0],out_channels=64,stride=1)
        self.layer2 = self.make_layer(block,layers[1],out_channels=128,stride=1)
        self.layer3 = self.make_layer(block,layers[2],out_channels=256,stride=1)
        self.layer4 = self.make_layer(block,layers[3],out_channels=512,stride=1)

        self.avgpool = nn.AdaptiveAvgPool2d((1,1))
        self.fc = nn.Linear(512*4,num_classes)

        self.softmax = nn.Softmax(dim=1)

    def forward(self,x):
        x=self.conv1(x)
        x=self.bn1(x)
        x=self.relu(x)
        x=self.maxpool(x)

        x=self.layer1(x)
        x=self.layer2(x)
        x=self.layer3(x)
        x=self.layer4(x)

        x=self.avgpool(x)
        x=x.reshape(x.shape[0],-1)
        x=self.fc(x)

        x = self.softmax(x)

        return x


    def make_layer(self,block,num_residual_blocks, out_channels, stride):
        identity_downsample = None
        layers = []

        if stride!=1 or self.in_channels != out_channels*4:
            identity_downsample= nn.Sequential(nn.Conv2d(self.in_channels,out_channels*4,kernel_size=1,stride=stride), nn.BatchNorm2d(out_channels*4))

        layers.append(block(self.in_channels,out_channels, identity_downsample,stride))
        self.in_channels = out_channels*4

        for i in range(num_residual_blocks -1):
            layers.append(block(self.in_channels,out_channels))
        return nn.Sequential(*layers)

In [ ]:
# Load the augmented data into a dataset
data_transforms_res = transforms.Compose([
    transforms.Resize(256), # Resize to a larger size to ensure all images are covered
    transforms.CenterCrop(224), # Crop the center of the image to 224x224
    transforms.ToTensor(),
])
combined_data_dir = '/content/drive/MyDrive/MP_Dataset/autism/combined_dataset'
combined_dataset = ImageFolder(combined_data_dir, transform=data_transforms_res)

# Define the lengths of the training and validation sets
train_size = len(combined_dataset)
validation_size = int(train_size * 0.2) # 20% of the data for validation
train_size = train_size - validation_size # Adjust the training size

# Split the dataset into training and validation sets
train_dataset, validation_dataset = random_split(combined_dataset, [train_size, validation_size])

# Create DataLoaders for the training and validation sets
train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=2)
validation_dataloader = DataLoader(validation_dataset, batch_size=8, shuffle=False, num_workers=2)

# Update the dataset_sizes dictionary to include the validation set
dataset_sizes = {
    'train': train_size,
    'validation': validation_size
}

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Check if the checkpoint file exists
model_save_path = '/content/drive/MyDrive/MP_Dataset/SpectrumSense/Model_checkpoints/ResNET50/Resnet_50_30epoch.pth'
checkpoint_exists = os.path.isfile(model_save_path)

# Initializing the model
model = ResNet(block, [3, 4, 6, 3], image_channels=3, num_classes=len(combined_dataset.classes))

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9, weight_decay=0.01)

# Load the checkpoint if it exists
if checkpoint_exists:
    checkpoint = torch.load(model_save_path, map_location=torch.device('cpu'))
    model.load_state_dict(checkpoint['model_state_dict'])
    # Move the model to the GPU
    model = model.to(device)
    # Move the optimizer's state dict to the GPU
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    for state in optimizer.state.values():
        for k, v in state.items():
            if isinstance(v, torch.Tensor):
                state[k] = v.to(device)
    last_epoch = checkpoint['epoch']
    last_loss = checkpoint['loss']
else:
    # If the checkpoint does not exist, initialize the last epoch and loss
    last_epoch = 0
    last_loss = 0.0

model = model.to(device)

num_epochs = 30

for epoch in range(last_epoch + 1, num_epochs + 1):
    print('Epoch {}/{}'.format(epoch, num_epochs))
    print('-' * 10)

    for phase in ['train', 'validation']:
        if phase == 'train':
            model.train()
        else:
            model.eval()

        running_loss = 0.0
        running_corrects = 0

        # Use the appropriate DataLoader based on the phase
        if phase == 'train':
            dataloader = train_dataloader
        else:
            dataloader = validation_dataloader

        for inputs, labels in dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()

            with torch.set_grad_enabled(phase == 'train'):
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)

                if phase == 'train':
                    loss.backward()
                    optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        epoch_loss = running_loss / dataset_sizes[phase]
        epoch_acc = running_corrects.double() / dataset_sizes[phase]

        print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

    # Save the model state after each epoch
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': epoch_loss,
        'accuracy': epoch_acc,
    }, model_save_path)

print('Training complete')

In [ ]:
model_save_path = '/content/drive/MyDrive/MP_Dataset/SpectrumSense/Model_checkpoints/ResNET50/Resnet_50_30epoch.pth'

# Path to the test image
test_image_path = '/content/drive/MyDrive/MP_Dataset/autism/test/autistic/025.jpg'


#ASK ME BEFORE RUNNING THIS CODE
data_transforms_test = transforms.Compose([
    transforms.Resize(256), # Resize to a larger size to ensure all images are covered
    transforms.CenterCrop(224), # Crop the center of the image to 224x224
    transforms.ToTensor(),
])


model = ResNet(block,[3, 4, 6, 3], image_channels=3, num_classes=len(combined_dataset.classes))
checkpoint = torch.load(model_save_path)
model.load_state_dict(checkpoint['model_state_dict'])
model.eval() # Set the model to evaluation mode
model.to('cuda' if torch.cuda.is_available() else 'cpu')

# Load and transform the test image
image = Image.open(test_image_path)
image = data_transforms_test(image).unsqueeze(0)

image = image.to(device)

# Make a prediction
with torch.no_grad():
    outputs = model(image)
    probabilities = F.softmax(outputs, dim=1)
    _, preds = torch.max(probabilities, 1)

class_names = ['Not Autistic', 'Autistic']

# Predicted class name
predicted_class = class_names[preds.item()]

# Print the probabilities of confidence rate for each class
print(f'Predicted class: {predicted_class}')
print(f'Probabilities: {probabilities.tolist()}')